In [36]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from imutils import paths
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [37]:
gestures_map = {'fist': 0,
               'L': 1,
               'okay': 2,
               'palm': 3,
               'peace': 4}

In [38]:
image_size = 224
image_path = 'data'

In [39]:
def process_image(path):
    img = Image.open(path)
    img = img.resize((image_size, image_size))
    img = np.array(img)
    return img

In [40]:
def process_data(X_data, y_data):
    X_data = np.array(X_data, dtype = 'float32')
    X_data = np.stack((X_data,)*3, axis = -1)
    X_data /= 255.0
    y_data = np.array(y_data)
    y_data = to_categorical(y_data)
    return X_data, y_data

In [41]:
def walk_through_file(image_path):
    X_data = []
    y_data = []
    impaths = list(paths.list_images(image_path))
    for impath in impaths:
        sign = impath[(len(image_path)+1):].split('_')[0]
        y_data.append(gestures_map[sign])
        X_data.append(process_image(impath))
    
    X_data, y_data = process_data(X_data, y_data)

    return X_data, y_data

In [42]:
X_data, y_data = walk_through_file(image_path)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.2, random_state=12, stratify=y_data)

In [45]:
checkpoint = ModelCheckpoint(filepath = 'hand_model.h5', save_best_only = True)
early_stopping = EarlyStopping(monitor = 'val_acc', min_delta = 0, patience = 10, verbose = 1, mode = 'auto', restore_best_weights = True)

In [46]:
model1 = VGG16(weights = 'imagenet', include_top = False, input_shape = (image_size, image_size, 3))
optimizer1 = 'adam'
base_model = model1

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [49]:
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
x = Dense(128, activation='relu', name='fc2a')(x)
x = Dense(128, activation='relu', name='fc3')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu', name='fc4')(x)
pred = Dense(5, activation = 'softmax')(x)
model = Model(inputs = base_model.input, outputs = pred)

for layer in base_model.layers:
    layer.trainable = False  

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), verbose=1,
          callbacks=[early_stopping, model_checkpoint])


model.save('mymodel.h5')